# Programação Dinamica

É básicamente uma técnica que torna possível resolver problemas que podem ser rescritos como combinação de problemas "menores". Como vocês vão ver, programação dinâmica em si é bem simples, o desafio mesmo é conseguir observar problemas como PDs.

#### Exemplo 1: Fibonacci

O problema de se determinar o $n$-ésimo número de fibonacci pode ser descrito em função de problemas menores, seja $F(n)$ a função que retorna o $n$-ésimo número de fibonacci, temos:

\begin{align}
F(n)=F(n-1)+F(n-2)\\
F(1)=F(0)=1
\end{align}

Nesse caso a relação "menor" é literalmente a relação $<$ entre dois números. Isso nem sempre acontece, o que importa mesmo é que não haja ciclos, isso vai ficar claro mais adiante.

Outra requisito importante é que sejam definidos casos base para a recorrência.

![](fibn.png)

Veja que a partir da recorrência, podemos definir um grafo. As arestas desse grafo são chamadas de transições e os vértices são os estados. Geralmente nos referimos a eles como estados e transições de uma PD.

A partir dessa recorrência também podemos implementar uma função que computa o $n$-ésimo fibonacci:

```c
int fib(int n) {
    if (n<1)
        return 1;
   
    return fib(n-1) + fib(n-2);
}
```

Fica como exercício mostrar a complexidade desse algoritmo que é de $O(F(n))=O(2^n)$ (existem bounds melhores para $F(n)$). 

## Memoização

Sim, está certo, não é memorização, é memoização mesmo!

O código que implementamos para calcular o fibonacci não é muito esperto. Simule ele para o exemplo da figura do grafo e veja que alguns estados são visitados MUITAS vezes. Isso pode ser melhorado se guardarmos a resposta de um estado uma vez que ele é computado.

Quando fazemos um DFS em um grafo, nunca visitamos um vertice mais de uma vez, simplismente usamos o valor calculado anteriormente.
Memoização é isso, armazenamos o valor de um estado e nunca o visitamos de novo. A implementação fica assim:

```c++
int dp[100];
bool vis[100];

int fib(int n){
    if (n<1) return 1;
    if (vis[n]) return dp[n];
    vis[n]=true;
    return dp[n]=f(n-1)+f(n-2);
}
```

A complexidade fica sendo a complexidade de uma DFS no grafo da PD: 

$O(|Estados| + |Transicoes|)$. Para o caso do fibonacci temos $O(n + 2n) = O(n)$.

Este jeito de se resolver uma PD é conhecido como top-down.

## Bottom-up

Uma outra maneira de se resolver uma PD é começando pelos casos base e iterando para os casos maiores. Essa maneira é conhecida como bottom-up ou iterativa.
Não existe apenas um jeito de se implementar uma PD iterativa, pessoalmente eu gosto de deixá-la bem parecida com a recorrência e setar separadamente os casos base:

```c++
int dp[100];
dp[0]=dp[1]=1;

for (int i=2; i<=n; ++i)
  dp[i]=dp[i-1]+dp[i-2];  
```

Uma dificuldade de se utilizar bottom-up é encontrar uma ordem correta para computar os estados, ou seja, como fazer o **for**. Para o fibonacci isso é bem simples.

É importante saber fazer bottom-up, algumas técnicas mais avançadas de otimização de PD só podem ser aplicadas utilizando bottom-up.

## Mais exemplos

Para ficar bom em resolver problemas de PD só tem um jeito: resolver MUITOS problemas de PD. Existem muitos tipos de problemas diferentes e é essencial saber, todo round do codeforces tem um quase.

Aqui vou mostrar os exemplos mais clássicos.

### Exemplo2: Coin change

Dado um conjunto $C=\{c_1,\dots,c_k\}$ de moedas, quero saber qual é o número mínimo de moedas para dar um troco de valor $X$. Posso usar cada moeda uma única vez.

Uma ideia para resolver esse problema é pensar na função $f(i, x)$ como sendo o troco mínimo para o valor $x$ utilizando as moedas $\{i, \dots, n\}$ (1-based). O caso base é $f(n+1, x)$, em que não há mais moedas para ser utilizada.

> case não-base:
\begin{align}
f(i, x) = \begin{cases}\min\{f(i+1, x-c_i)+1, f(i+1, x)\} & c_i \le x\\
            f(i+1, x) & c_i > x
\end{cases}
\end{align}

> caso base:
\begin{align}
f(n+1,x) = \begin{cases}0 & x=0\\ \infty & x>0 \end{cases}
\end{align}

##### Implementação

```c++
int dp[100][5000]; //inicialize com -1 para indicar que nao foi visitado
int c[100]; // conjunto de moedas, valor de cada moeda é menor do que 50

int f(int i, int x) {
    if (i==n)
        return (x==0) ? 0 : INF;
    if (x < c[i]) return dp[i][x] = f(i+1, x);
    return dp[i][x] = min(f(i+1, x), f(i+1, x-c[i])+1);
}
```
Veja que a complexidade é $O(|C| \times K)$ em que $K$ é o valor máximo do troco.

##### Exercícios:

- Caso em que cada moeda pode ser usada mais de uma vez
- Implementação bottom-up
- Substituir o estado que representa o índice da moeda por um **for**
- Contar quantas soluções ótimas existem

#### Memória

É hora de começar a se preocupar com memória. No codeforces, por exemplo, conseguimos executar $~10^8$ operações em $1s$. No entanto, armazenar um vetor de $10^8$ inteiros requer aproximadamente 500mb, se vc tentar fazer isso, provavelmente vai tomar MLE.

No problema da moedas se $|C|=100$ e $K=10^6$ poderíamos fazer um trick na implementação bottom-up do problema para não precisar reservar 10^8 de memória. Veja:

```c++
dp[2][1000001];
dp[1][0]=0;
K=1000000;
for(int i = 1; i<=K; i++) dp[1][K]=INF;

int cur = 0, prv=1;
for(int i = 0; i<k; i++)//k é o número de moedas
{
    dp[cur][0]=0;
    for (int j = 1; j<=c[k];j--){//c[k] é a k-esima moeda
        dp[cur][j]=INF;
        dp[cur][j]=min(dp[cur][j], dp[prv][j-c[k]]+1)
    }
    
    swap(cur,prv);
}

cout << dp[prv][X] << endl;
```

Para entender o trick de memória, primeiro implemente a versão bottom-up que não precisa economizar memória. Você vai perceper que a linha $i$ da sua matriz só precisa da linha $i-1$ para calcular seu valor, logo só precisamos das últimas duas linhas na dp.

### Longest Common Subsequence (LCS)

Sejam $A=\{a_1, \dots, a_n\}$ e $B=\{b_1, \dots, b_n\}$ duas strings, queremos saber qual é a subsequencia mais longa em comum nas duas. Uma subsequência é formada a partir de uma sequencia eliminando alguns elementos e mantendo sua ordem. Por exemplo:

\begin{align}
A=abcabcabd\\
B=axcxxcyby
\end{align}

A maior subsequencia comum entre $A$ e $B$ é $accb$.
